In [1]:
from transformers import DistilBertForZeroShotClassification

ImportError: cannot import name 'DistilBertForZeroShotClassification' from 'transformers' (c:\Users\hello\anaconda3\Lib\site-packages\transformers\__init__.py)

In [ ]:
def lig_color_map(predict, ):

    lig = LayerIntegratedGradients(predict, self.model.distilbert.embeddings)


In [ ]:
def lig_color_map_qanda(self):

    lig = LayerIntegratedGradients(self.predict, self.model.distilbert.embeddings)

    attributions_start, delta_start = lig.attribute(inputs=self.preprocessor.input_ids,
                                                    baselines=self.preprocessor.baseline_input_ids,
                                                    additional_forward_args=(self.preprocessor.attention_mask,0,False),
                                                    return_convergence_delta=True)
    attributions_end, delta_end = lig.attribute(inputs=self.preprocessor.input_ids, 
                                                baselines=self.preprocessor.baseline_input_ids,
                                                additional_forward_args=(self.preprocessor.attention_mask,1,False),
                                                return_convergence_delta=True)

    attributions_start_sum = summarize_attributions(attributions_start)
    attributions_end_sum = summarize_attributions(attributions_end)

    # storing couple samples in an array for visualization purposes
    start_position_vis = viz.VisualizationDataRecord(
                            attributions_start_sum,
                            torch.max(torch.softmax(self.start_scores[0], dim=0)),
                            torch.argmax(self.start_scores),
                            torch.argmax(self.start_scores),
                            str(self.preprocessor.ground_truth_start_ind),
                            attributions_start_sum.sum(),       
                            self.preprocessor.all_tokens,
                            delta_start)

    end_position_vis = viz.VisualizationDataRecord(
                            attributions_end_sum,
                            torch.max(torch.softmax(self.end_scores[0], dim=0)),
                            torch.argmax(self.end_scores),
                            torch.argmax(self.end_scores),
                            str(self.preprocessor.ground_truth_end_ind),
                            attributions_end_sum.sum(),       
                            self.preprocessor.all_tokens,
                            delta_end)

    print('\033[1m', 'Visualizations For Start Position', '\033[0m')
    viz.visualize_text([start_position_vis])

    print('\033[1m', 'Visualizations For End Position', '\033[0m')
    viz.visualize_text([end_position_vis])

In [ ]:
def lig_color_map_sentiment(self):
    
    lig = LayerIntegratedGradients(self.predict, self.model.distilbert.embeddings)
    
    # Get attributions for the sentiment prediction
    attributions, delta = lig.attribute(inputs=self.preprocessor.input_ids,
                                        baselines=self.preprocessor.baseline_input_ids,
                                        additional_forward_args=(self.preprocessor.attention_mask, False),
                                        return_convergence_delta=True,
                                        target=self.preprocessor.ground_truth_index)
    
    attributions_sum = summarize_attributions(attributions)
    
    # Use the stored logits
    sentiment_score = torch.softmax(self.logits[0], dim=0)
    predicted_class = torch.argmax(sentiment_score)
    
    sentiment_vis = viz.VisualizationDataRecord(
                        attributions_sum,
                        torch.max(sentiment_score),
                        predicted_class,
                        predicted_class,
                        str(self.preprocessor.ground_truth_index),
                        attributions_sum.sum(),       
                        self.preprocessor.all_tokens,
                        delta)

    print('\033[1m', 'Visualizations For Sentiment Prediction', '\033[0m')
    viz.visualize_text([sentiment_vis])
  
  def lig_top_k_tokens(self, k:int=5) -> None:
    lig = LayerIntegratedGradients(self.predict, [self.model.distilbert.embeddings.word_embeddings])

    attributions = lig.attribute(inputs=self.preprocessor.input_ids,
                                 baselines=self.preprocessor.baseline_input_ids,
                                 additional_forward_args=(self.preprocessor.attention_mask,False),
                                 target=self.preprocessor.ground_truth_index)

    attributions_word = summarize_attributions(attributions[0])
    top_words, top_words_val, top_word_ind = get_top_k_attributed_tokens(attributions_word, k=k, preprocessor=self.preprocessor)

    df = pd.DataFrame({'Word(Index), Attribution': ["{} ({}), {}".format(word, pos, round(val.item(),2)) for word, pos, val in zip(top_words, top_word_ind, top_words_val)]})
    df.style.set_properties(cell_ids=False)

    full_token_list = ['{}({})'.format(token, str(i)) for i, token in enumerate(self.preprocessor.all_tokens)]

    print(f"Full token list: {full_token_list}")
    print(f"Top {k} attributed embeddings for sentiment prediction: {df}")